## 문제 6

**Kaggle 형** train_prob.csv로 문제 target을 예측하는 모델을 만들고, 

test_prob.csv에 대한 target 예측하여 다음과 같은 형식의 answer6.csv를 만들어라.

id, target

0, 6.9

5, 7.8

...


**평가지표**

$RMSE(Y, \hat{Y}) = \sqrt{\frac{1}{n}\sum^{n}_{i=1}(y_i-\hat{y_i})^2}$

**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col=['id'])
df_test = pd.read_csv('test_prob.csv', index_col=['id'])
df_ans = pd.read_csv('test_prob_ans.csv', index_col=['id'])

In [3]:
# 처리 과정에 필요하 내용들을 list 형태로 구성합니다.
repl_list = [
    ('cat3', {"B": "C"}, [83634, 147361, 9005]),
    ('cat4', {"A": "B", "D": "B"}, [239397, 603]), 
    ('cat6', {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'}, [234203, 5145, 652]),
    ('cat7', {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'}, [4606, 19784, 214027, 1583]),
    ('cat8', {'B': 'G', 'F': 'E'}, [30338, 96743, 2953, 76085, 33881]),
    ('cat9', {'C': 'H', 'D': 'B', 'E': 'L'}, [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968])
]
# 반복문 처리 내용들을 수행합니다.
for c, d, cnt in repl_list:
    print(c, d, cnt)
    # 치환후 내용을 s_repl에 저장합니다
    s_repl = df_train[c].replace(d)
    # 치환결과를 확인합니다.
    if not (s_repl.nunique() == len(cnt) and (s_repl.value_counts().sort_index() == cnt).all()):
        print("Error", c)
        break
    df_train[c] = s_repl
    df_test[c] = df_test[c].replace(d)

cat3 {'B': 'C'} [83634, 147361, 9005]
cat4 {'A': 'B', 'D': 'B'} [239397, 603]
cat6 {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'} [234203, 5145, 652]
cat7 {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'} [4606, 19784, 214027, 1583]
cat8 {'B': 'G', 'F': 'E'} [30338, 96743, 2953, 76085, 33881]
cat9 {'C': 'H', 'D': 'B', 'E': 'L'} [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968]


In [4]:
# 문제 4번을 활용하기 위해 만듭니다.
df_train['targetA'] = df_train['target'] <= 7.45

In [5]:
from scipy.stats import norm

df_train_clf = df_train.assign(
    # H0: target은 A 범주, H1: target B 범주, target ~ N(mu = 6.769, std = 0.616) 우측 꼬리
    prob_A = 1 - norm.cdf(df_train['target'], loc=6.769, scale=0.616),
    # H0: target은 B 범주, H1: target A 범주, target ~ N(mu = 8.123, std= 0.527) 좌측 꼬리
    prob_B = norm.cdf(df_train['target'], loc=8.123, scale=0.527)
)
df_train_clf = df_train_clf.query('prob_B < 0.01 or prob_A < 0.01').copy()

In [6]:
# 문제 4번 모델을 만듭니다. targetA일 확률을 활용할 예정입니다.
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(), ['cat{}'.format(i) for i in range(10)]),
    ('pt', 'passthrough', ['cont{}'.format(i) for i in range(14)])
])
X_xgb = ['cont{}'.format(i) for i in range(14)] + ['cat{}'.format(i) for i in range(10)]
clf_xgb = make_pipeline(
    ct,
    xgb.XGBClassifier(
        max_depth = 2, # 트리의 최대 깊이 2
        reg_alpha = 0.1, # L1 규제 0.1
        reg_lambda = 0.1, # L2 규제 0.1
        colsample_bytree=0.25, # 트리 당 컬럼 샘플링 비율 0.25
        n_estimators=500, # 트리의 수 500
        random_state=123, # random_state 123
    )
)

X_xgb = ['cat{}'.format(i) for i in range(10)] + ['cont{}'.format(i) for i in range(14)]
clf_xgb.fit(df_train_clf[X_xgb], df_train_clf['targetA'])
df_train['targetA_prob'] = clf_xgb.predict_proba(df_train[X_xgb])[:, 1]
df_test['targetA_prob'] = clf_xgb.predict_proba(df_test[X_xgb])[:, 1]

In [7]:
q = [i for i in np.arange(0, 1.01, 0.01)]
# 나머지 변수에 대해서도 해당 파생 변수를 만들어 줍니다.
for i in range(0, 14):
    col = 'cont{}'.format(i)
    q_val = df_train[col].quantile(q)
    q_val.iloc[[0, -1]] = [-np.inf, np.inf]
    q_range = pd.cut(df_train[col], bins=q_val)
    q_mean = df_train.groupby(q_range)['target'].mean()
    df_train[col + '_q'] = q_range.map(q_mean).astype('float')
    df_test[col + '_q'] = pd.cut(df_test[col], bins=q_val).map(q_mean).astype('float')

In [8]:
# 공통
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import ShuffleSplit, KFold 

cv = KFold(n_splits=5, random_state=123)
# train(80%)/test(20%) 한 번으로 검증합니다. XGB, RF등 오래 걸리는 모델을 위해 사용합니다.
ss = ShuffleSplit(n_splits=1, train_size=0.8, random_state=123)

df_ans = pd.read_csv('test_prob_ans.csv', index_col='id')
X = df_test.columns.tolist() + ['cont{}_q'.format(i) for i in range(14)] + ['targetA_prob']

cat_cols = ['cat{}'.format(i) for i in range(10)]
cont_cols = ['cont{}'.format(i) for i in range(14)]
cont_q_cols = ['cont{}_q'.format(i) for i in range(14)]

# 위에서 발생한 leak을 바로 잡아 교차검증을 합니다.
q = [i for i in np.arange(0, 1.01, 0.01)]
def eval_model(model, sp):
    score_train, score = list(), list()
    for train_idx, test_idx in sp.split(df_train):
        df_cv_train, df_cv_test = df_train.iloc[train_idx].copy(), df_train.iloc[test_idx].copy()
        # 검증셋에서 train으로 파생변수를 만들고
        # 검증셋의 test(겹외셋)에 검증셋의 train으로 만든 통계값(mean)을 반영합니다.
        for i in range(0, 14):
            col = 'cont{}'.format(i)
            qt = df_cv_train[col].quantile(q)
            qt.iloc[[0, -1]] = [-np.inf, np.inf]
            q_range = pd.cut(df_cv_train[col], bins=qt)
            q_mean = df_cv_train.groupby(q_range)['target'].mean()
            df_cv_train[col + '_q'] = q_range.map(q_mean).astype('float')
            df_cv_test[col + '_q'] = pd.cut(df_cv_test[col], bins=qt).map(q_mean).astype('float')
        model.fit(df_cv_train[X], df_cv_train['target'])
        score_train.append(-(mean_squared_error(df_cv_train['target'], model.predict(df_cv_train[X]))) ** 0.5)
        score.append(-(mean_squared_error(df_cv_test['target'], model.predict(df_cv_test[X]))) ** 0.5)
    return score_train, score

def choose_model(model):
    model.fit(df_train[X], df_train['target'])
    prd = model.predict(df_test[X])
    pd.DataFrame({
        'id': df_test.index.values,
        'target': prd
    }).to_csv('answer6.csv', index = None)
    return prd 

In [9]:
from sklearn.linear_model import LinearRegression

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), cat_cols),
    ('std', StandardScaler(), cont_cols)
])

reg_lr = make_pipeline(ct, LinearRegression())
result = eval_model(reg_lr, cv)

In [10]:
np.mean(result[1]), np.std(result[1]), np.mean(result[0])

(-0.8632456423386847, 0.0029474142614306633, -0.8630025788522048)

In [11]:
# baseline 모델 선택
prd = choose_model(reg_lr)
# 자가 채점
mean_squared_error(df_ans['target'], prd) ** 0.5

0.8657267201878257

In [12]:
# + targetA_prob 를 추가합니다.

from sklearn.linear_model import LinearRegression

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), cat_cols),
    ('std', StandardScaler(), cont_cols),
    ('pt', 'passthrough', ['targetA_prob'])
])
reg_lr_2 = make_pipeline(
    ct, 
    LinearRegression()
)
result = eval_model(reg_lr_2, cv)
np.mean(result[1]), np.std(result[1]), np.mean(result[0])

(-0.8435148589687671, 0.003012400908502579, -0.8432679868397729)

In [13]:
# 개선이 있습니다. 모델을 선택합니다.
prd = choose_model(reg_lr_2)
# 자가 채점을 해봅니다.
mean_squared_error(df_ans['target'], prd) ** 0.5

0.8493184210459026

In [14]:
# + cont{}_q 를 cont{} 대신에 사용합니다.

from sklearn.linear_model import LinearRegression

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), cat_cols),
    ('std', StandardScaler(), cont_q_cols),
])
reg_lr_3 = make_pipeline(
    ct, 
    LinearRegression()
)
result = eval_model(reg_lr_3, cv)
np.mean(result[1]), np.std(result[1]), np.mean(result[0])

(-0.8486955888442786, 0.0029782918718652706, -0.8446443969137316)

In [15]:
# + targetA_prob 를 추가합니다.
# + cont{}_q 를 cont{} 대신에 사용합니다.

from sklearn.linear_model import LinearRegression

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), cat_cols),
    ('std', StandardScaler(), cont_q_cols),
    ('pt', 'passthrough', ['targetA_prob'])
])
reg_lr_4 = make_pipeline(
    ct, 
    LinearRegression()
)
result = eval_model(reg_lr_4, cv)
np.mean(result[1]), np.std(result[1]), np.mean(result[0])

(-0.8436309067028711, 0.0029638364083831997, -0.8414521963496469)

In [16]:
# 개선이 있습니다. 모델을 선택합니다.
prd = choose_model(reg_lr_4)
# 자가 채점을 해봅니다.
mean_squared_error(df_ans['target'], prd) ** 0.5

0.8479039208644088

In [17]:
# + targetA_prob 를 추가합니다.
# PCA + cont{}_q 를 cont{} 대신에 사용합니다.

from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), cat_cols),
    ('std', make_pipeline(StandardScaler(), PCA(n_components=0.95)), cont_q_cols),
    ('pt', 'passthrough', ['targetA_prob'])
])
reg_lr_5 = make_pipeline(
    ct, 
    LinearRegression()
)
result = eval_model(reg_lr_5, cv)
np.mean(result[1]), np.std(result[1]), np.mean(result[0])

(-0.8436397849645798, 0.0029162200124757966, -0.8414205062520068)

In [18]:
from sklearn.ensemble import RandomForestRegressor

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(), cat_cols),
    ('pt', 'passthrough', cont_cols + ['targetA_prob'])
])

reg_rf = make_pipeline(
    ct,
    RandomForestRegressor(n_estimators=50, max_depth=4, random_state=123, n_jobs=4)
)
# RandomForestRegressor는 학습에 오래걸립니다. 
# 검증 방법을 교차검증에서 Hold Out (Shuffle Split n_splits=1) 검증으로 바꿉니다.
result = eval_model(reg_rf, ss)
np.mean(result[1]), np.mean(result[0])

(-0.8432410423704046, -0.8430963969319141)

In [19]:
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(), cat_cols),
    ('pt', 'passthrough', cont_cols + ['targetA_prob'])
])
reg_xgb = make_pipeline(
    ct,
    xgb.XGBRegressor(n_estimators=150, max_depth=2, random_state=123, n_jobs=4)
)
# xgboost도 역시 오래 걸립니다. 
# Hold Out (Shuffle Split n_splits=1) 검증을 사용합니다.
result = eval_model(reg_xgb, ss)
np.mean(result[1]), np.mean(result[0])

(-0.8426761026271767, -0.8414261158818475)

In [20]:
# Voting을 통한 앙상블을 만듭니다.
from sklearn.ensemble import VotingRegressor

reg_vt = VotingRegressor([
    #('lr_1', reg_lr),
    #('lr_2', reg_lr_2),
    #('lr_3', reg_lr_3),
    ('lr_4', reg_lr_4),
    #('lr_5', reg_lr_5),
    ('rf', reg_rf),
    ('xgb', reg_xgb),
])
# 시간이 덜 걸리는 Hold Out 검증을 사용하여 탐색 후,
# 선정한 모델을을 최종의로 교차 검증을 통해 보다 정교하게 성능을 측정합니다
result = eval_model(reg_vt, cv)
np.mean(result[1]), np.std(result[1]), np.mean(result[0])

(-0.8427622265063368, 0.0029545668334796946, -0.8413866017389567)

In [21]:
# 개선이 있습니다.(xgb 보다 신뢰할 만한 교차 검증을 통한 결과이니, 이를 감안합니다.) 모델을 선택합니다. 
prd = choose_model(reg_vt)
# 자가 채점을 해봅니다.
mean_squared_error(df_ans['target'], prd) ** 0.5

0.8478204965772116

Kaggle 형의 만점 기준은 **0.849** 이하입니다

3일 간의 실습 강의를 포함하여 총 8일간 강의 들으시느라 고생 많으셨습니다.

궁금하신점 있으시면 연락주세요.

좋은 소식이 들려 오기를 기다리겠습니다.

감사합니다.

**멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)** 올림